Инициализация

In [3]:
import numpy as np
import tensorflow as tf
import spektral

Cora - набор данных для обучения представлению графов (потом надо найти инфу по ней из видоса)
spektral позволяет нам быстро загружать и предварительно обрабатывать многие стандартные наботы данных для обучения представлению графов (к примеру, Cora). Она поставляется с полезными функциями загрузки, которые позволят нам прямой доступ к таким элементам, как:
adj - матрица смежности графа
features - матрица функций, которая дает нам функцию в каждом из узлов
labels - метки, обозначающие тему каждой статьи
train_mask - массив масок: какие узлы принадлежат обучающему набору (training set)
val_mask - массив масок: какие узлы принадлежат проверочному набору (validation set)
test_mask - массив масок: какие узлы принадлежат тестовому набору (test set)

Загружаем эти данные из набора данных 'Cora'

In [4]:
cora_dataset = spektral.datasets.citation.Citation(name='Cora')
test_mask = cora_dataset.mask_te
train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
graph = cora_dataset.graphs[0]
features = graph.x
adj = graph.a
labels = graph.y

#features = features.todense()
adj = adj + np.eye(adj.shape[0])
#features = features.astype('float32')
#adj = adj.astype('float32')

adj_np = np.asarray(adj, np.float32)
adj_tf = tf.convert_to_tensor(adj_np, np.float32)

features_np = np.asarray(features, np.float32)
features_tf = tf.convert_to_tensor(features_np, np.float32)

print("Type of adj:", type(adj_tf))
print("Type of features:", type(features_tf))

#Выведем размеры набора данных Cora
print(graph)
print(features_tf.shape)
print(adj_tf.shape)
print(labels.shape)

#Выведем количество узлов в каждом наборе
print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

Type of adj: <class 'tensorflow.python.framework.ops.EagerTensor'>
Type of features: <class 'tensorflow.python.framework.ops.EagerTensor'>
Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(2708, 1433), dtype=float32)
tf.Tensor(
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]], shape=(2708, 2708), dtype=float32)
(2708, 7)
140
500
1000


Опредем функцию потери перекрестной энтропии и посчета точности вычисления для конкретного набора (mask)

In [5]:
def masked_softmax_cross_entropy(logits, labels, mask):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)

def masked_accuracy(logits, labels, mask):
    corrent_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(corrent_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

Определим слой GNN
fts - матрица признаков узлов
adj - матрица смежности
transform - некоторая трансформация, которая будет применяться к каждому узлу
activation - функция активации

In [6]:
def gnn(fts, adj, transform, activation):
    seq_fts = transform(fts) #эквивалент матрицы весов W
    #if isinstance(adj, tf.sparse.SparseTensor):
    #    ret_fts = tf.sparse.sparse_dense_matmul(adj, seq_fts)
    #else:
    ret_fts = tf.matmul(adj, seq_fts)
    return activation(ret_fts)

# def gnn(fts, adj, weights, activation):
    
#     # Check if adj is a sparse matrix
#     if isinstance(adj, tf.sparse.SparseTensor):
#         print("adj is a sparse matrix")
#     else:
#         print("adj is not a sparse matrix")
        
#     # Check if seq_fts is a sparse matrix
#     if isinstance(fts, tf.sparse.SparseTensor):
#         print("fts is a sparse matrix")
#     else:
#         print("fts is not a sparse matrix")

#     if isinstance(adj, tf.sparse.SparseTensor):
#         ret_fts = tf.sparse.sparse_dense_matmul(adj, fts)
#     else:
#         ret_fts = tf.matmul(adj, fts)

#     print("Type of ret_fts = ", type(ret_fts))
#     print("Type of weights = ", type(weights))
#     return activation(tf.matmul(ret_fts, weights))

Определим двуслойную GNN для обучения на данных Cora
fts - матрица признаков узлов
adj - матрица смежности
gnn_fn - некоторая модель GNN 
units - количество единиц, которые будет вычислять gnn в каждом узле (сколько изменений в скрытых функциях)
epochs - количество эпох обучения
lr - скорость обучения


In [7]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
    layer_1 = tf.keras.layers.Dense(units, activation=tf.nn.relu) #скрытый слой равный числу units
    layer_2 = tf.keras.layers.Dense(7, activation=tf.nn.relu) #определяет классификацию каждого узла

    def cora_gnn(fts, adj):
        hidden = gnn_fn(fts, adj, layer_1, tf.nn.relu) #tf.nn.relu - нелинейная функция активации
        logits = gnn_fn(hidden, adj, layer_2, tf.identity)
        return logits
    
    optimazer = tf.keras.optimizers.Adam(learning_rate=lr)

    best_accuracy = 0.0
    for ep in range(epochs + 1):
        with tf.GradientTape() as tape:
            logits = cora_gnn(fts, adj)
            loss = masked_softmax_cross_entropy(logits, labels, train_mask)
        
        variables = tape.watched_variables()
        grads = tape.gradient(loss, variables)
        optimazer.apply_gradients(zip(grads, variables))

        logits = cora_gnn(fts, adj)
        val_accuracy = masked_accuracy(logits, labels, val_mask)
        test_accuracy = masked_accuracy(logits, labels, test_mask)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            print('Epoch', ep, '| Training loss:', loss.numpy(), '| Val accuracy:', val_accuracy.numpy(), '| Test accuracy:', test_accuracy.numpy())

Главный вызов обучения GNN

In [8]:
train_cora(features_tf, adj_tf, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 4.646867 | Val accuracy: 0.34999996 | Test accuracy: 0.352
Epoch 4 | Training loss: 3.7570548 | Val accuracy: 0.364 | Test accuracy: 0.355
Epoch 5 | Training loss: 2.4916403 | Val accuracy: 0.528 | Test accuracy: 0.50600004
Epoch 6 | Training loss: 1.584272 | Val accuracy: 0.618 | Test accuracy: 0.595
Epoch 7 | Training loss: 1.2613305 | Val accuracy: 0.634 | Test accuracy: 0.631
Epoch 8 | Training loss: 1.154107 | Val accuracy: 0.65 | Test accuracy: 0.65599996
Epoch 9 | Training loss: 1.0506023 | Val accuracy: 0.666 | Test accuracy: 0.67399997
Epoch 10 | Training loss: 0.9747684 | Val accuracy: 0.684 | Test accuracy: 0.69899994
Epoch 12 | Training loss: 0.8535274 | Val accuracy: 0.69 | Test accuracy: 0.70799994
Epoch 13 | Training loss: 0.7868511 | Val accuracy: 0.69200003 | Test accuracy: 0.714
Epoch 14 | Training loss: 0.7217266 | Val accuracy: 0.694 | Test accuracy: 0.706
Epoch 21 | Training loss: 0.5119682 | Val accuracy: 0.696 | Test accuracy: 0.705
Epoch

Немного изменим матрицу смежности графа, чтобы получить более устойчивую модель обучения

In [9]:
deg = tf.reduce_sum(adj_tf, axis=-1) #матрица степеней вершин графа
train_cora(features_tf, adj_tf / deg, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 1.9445586 | Val accuracy: 0.22599998 | Test accuracy: 0.213
Epoch 1 | Training loss: 1.830808 | Val accuracy: 0.35199997 | Test accuracy: 0.358
Epoch 2 | Training loss: 1.6829454 | Val accuracy: 0.45199996 | Test accuracy: 0.465
Epoch 3 | Training loss: 1.5160421 | Val accuracy: 0.516 | Test accuracy: 0.53
Epoch 4 | Training loss: 1.3495936 | Val accuracy: 0.59 | Test accuracy: 0.607
Epoch 5 | Training loss: 1.1923124 | Val accuracy: 0.644 | Test accuracy: 0.67399997
Epoch 6 | Training loss: 1.04554 | Val accuracy: 0.686 | Test accuracy: 0.70799994
Epoch 7 | Training loss: 0.91327184 | Val accuracy: 0.69 | Test accuracy: 0.717
Epoch 8 | Training loss: 0.79829204 | Val accuracy: 0.702 | Test accuracy: 0.733
Epoch 9 | Training loss: 0.7007021 | Val accuracy: 0.70600003 | Test accuracy: 0.745
Epoch 10 | Training loss: 0.61963934 | Val accuracy: 0.71199995 | Test accuracy: 0.747


Воспользуемся методом нормализации матрицы смежности, предложенную Томасом Кипфом в сетевой модели свертки графов

In [10]:
norm_deg = tf.linalg.diag(1.0 / tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg, tf.matmul(adj_tf, norm_deg))
train_cora(features_tf, norm_adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 1.9538654 | Val accuracy: 0.37199998 | Test accuracy: 0.38199997
Epoch 1 | Training loss: 1.8432275 | Val accuracy: 0.422 | Test accuracy: 0.431
Epoch 2 | Training loss: 1.7182083 | Val accuracy: 0.50600004 | Test accuracy: 0.50999993
Epoch 3 | Training loss: 1.5649697 | Val accuracy: 0.604 | Test accuracy: 0.58299994
Epoch 4 | Training loss: 1.3862036 | Val accuracy: 0.656 | Test accuracy: 0.657
Epoch 5 | Training loss: 1.1970687 | Val accuracy: 0.72199994 | Test accuracy: 0.709
Epoch 6 | Training loss: 1.0130662 | Val accuracy: 0.736 | Test accuracy: 0.73899996
Epoch 7 | Training loss: 0.84018743 | Val accuracy: 0.754 | Test accuracy: 0.754
Epoch 8 | Training loss: 0.6831143 | Val accuracy: 0.77 | Test accuracy: 0.76400006
Epoch 9 | Training loss: 0.5458493 | Val accuracy: 0.77199996 | Test accuracy: 0.7839999
Epoch 10 | Training loss: 0.42996103 | Val accuracy: 0.78 | Test accuracy: 0.7929999
Epoch 13 | Training loss: 0.20034412 | Val accuracy: 0.782 | Test 